In [1]:
# import
import csv
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from enum import Enum

# B. 결측치/이상치 처리 (실무의 70%)

### B1. 결측치 지도 만들기

- **문제:** 컬럼별 결측치 비율을 내림차순 정렬하여 상위 5개 컬럼을 구하고, 결측치 비율이 0인 컬럼 목록도 따로 구하세요.
    
- **무엇을 묻는가:** 결측치 진단, 정렬, 조건 필터링.
    

### B2. Age 결측치 “그룹 기반”으로 채우기

- **문제:** Age 결측치를 `Pclass`와 `Sex` 그룹의 중앙값으로 채우세요. 채우기 전/후 Age의 결측치 개수와 기술통계를 비교하세요.
    
- **무엇을 묻는가:** `groupby().transform('median')` 패턴, `fillna`, 전후 비교.
    

### B3. Fare 이상치 탐지 및 처리 전략 제안

- **문제:** Fare의 상위 1%를 이상치 후보로 보고, 해당 행을 추출하세요. 이후 (1) 그대로 둔다, (2) 로그 변환, (3) winsorize(상한 절단) 중 한 가지로 처리한 새 컬럼을 만드세요.
    
- **무엇을 묻는가:** 분위수 계산, 조건 필터, 파생 컬럼 생성, 변환 설계 감각.

In [2]:
import pandas as pd
from pathlib import Path
import os

df_raw = pd.read_csv(
    Path(os.getcwd()).parent.parent
    / "data"
    / "raw"
    / "kaggle"
    / "datasets"
    / "titanic"
    / "titanic"
    / "train.csv"
)
df_train = df_raw.copy()

In [3]:
# B - 1
s_na_ratio = df_train.isna().mean()
s_na_ratio_nonzero = s_na_ratio[s_na_ratio > 0].sort_values(ascending=False).head(5)
s_na_ratio_zero = s_na_ratio[s_na_ratio == 0]
pd.DataFrame({"top5_nonzero": s_na_ratio_nonzero})
pd.DataFrame({"na_ratio": s_na_ratio_zero})

,na_ratio
PassengerId,0.0
Survived,0.0
Pclass,0.0
Name,0.0
Sex,0.0
SibSp,0.0
Parch,0.0
Ticket,0.0
Fare,0.0


In [4]:
mask = df_train["Age"].isna()
group_median = df_train.groupby(["Pclass", "Sex"])["Age"].transform("median")

df_train.loc[mask, "Age"] = group_median.loc[mask]

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
q99 = df_train["Fare"].quantile(0.99)  # 상위 1% 컷
mask_out = df_train["Fare"] >= q99  # "상위 1%"를 포함(=)할지 여부는 정책

outlier_rows = df_train.loc[
    mask_out, ["PassengerId", "Fare", "Pclass", "Sex", "Survived"]
].copy()
outlier_rows

,PassengerId,Fare,Pclass,Sex,Survived
27,28,263.0000,1,male,0
88,89,263.0000,1,female,1
258,259,512.3292,1,female,1
311,312,262.3750,1,female,1
341,342,263.0000,1,female,1
438,439,263.0000,1,male,0
679,680,512.3292,1,male,1
737,738,512.3292,1,male,1
742,743,262.3750,1,female,1


In [6]:
df_train["Fare_proc"] = df_train["Fare"]
df_train["Fare_proc"] = np.log1p(df_train["Fare"])
df_train["Fare_proc"] = df_train["Fare"].clip(upper=q99)
df_train[["Fare", "Fare_proc"]].describe(percentiles=[0.95, 0.99, 0.999])

,Fare,Fare_proc
count,891.000000,891.000000
mean,32.204208,31.224767
std,49.693429,42.524125
min,0.000000,0.000000
50%,14.454200,14.454200
95%,112.079150,112.079150
99%,249.006220,247.669342
99.9%,512.329200,249.006220
max,512.329200,249.006220
